This program reads a tab separated file containing the author name affiliation, and generate the coordinates corresponding to their institude.

One line of the file looks like this:     
`Adam	Y.	Université de Liège	Liège	Belgium`      
The city column was added to make easier the localisation.

For each year, two output files are created:
1. a file storing only the coordinates, that can be used directly with leaflet (see [leaflet](./leaflet) folder) 
2. a file that is the same as the input file but with 2 extra columns for the coordinates.

If the coordinates cannot be found for some reasons, a fill value (e.g. -999) is used, and the coordinates will have to be found "manually", i.e., using other resources.

In [7]:
import os
import glob
import logging
from collections import Counter
from liegecolloquium import Participant
from importlib import reload

# Configuration

In [8]:
datadir = "../data/processed/"
outputdir = "../data/test2delete/"
logger = logging.getLogger()
logger.setLevel(logging.INFO)

In [9]:
filelist = sorted(glob.glob(os.path.join(datadir, "ParticipantList-197*")))
logging.info("Working on {0} file(s)".format(len(filelist)))

INFO:root:Working on 10 file(s)


In [10]:
participantlist = []
locationlist = []
if not(os.path.exists(outputdir)):
    os.makedirs(outputdir)
    
# Write full file witch coordinates and institute
with open(coordfile, 'w') as cfall:
    cfall.write("var coords = [\n")

for datafiles in filelist:
    np = 0
    logging.info("Working on file {0}".format(datafiles))
    outputbasename = os.path.basename(datafiles).replace(".tsv", ".dat")
    coordbasename = os.path.basename(datafiles).replace(".tsv", ".coord")
    outputfile = os.path.join(outputdir, outputbasename)
    coordfile = os.path.join(outputdir, coordbasename)
    logging.info("Writing info in file {0}".format(outputfile))
    logging.info("Writing coordinates in file {0}".format(outputfile))
    logging.info("----------------")
    
    with open(coordfile, 'w') as cf:
        cf.write("var coords = [\n")

    with open(datafiles, 'r') as f:
        for line in f:
            
            if line == '\n':
                logger.debug("Empty line")
            else:
                np += 1
                l = line.rstrip().split('\t')
                participant = Participant(l[0], l[1], l[2], l[3], l[4])
                logger.debug("Working with {0}".format(participant))

                # Modify country name via dictionary
                participant.replace_country()

                # Find location ; check if in the list before using geopy
                loc1 = ", ".join((participant.affiliation, 
                                  participant.city, 
                                  participant.country))
                
                # logger.debug(loc1)

                if loc1 in locationlist:
                    logger.debug("Already in the list")
                    ind = locationlist.index(loc1)
                    logger.debug("Position in the list: {0}".format(ind))
                    participant.lat = participantlist[ind].lat
                    participant.lon = participantlist[ind].lon
                    participant.countryname = participantlist[ind].countryname
                    locationlist.append(locationlist.index(loc1))
                else:
                    logger.debug("Not in the list, use geolocator to get coordinates")
                    locationstring, location = participant.get_location()
                    locationlist.append(locationstring)

                # Write info to a text file
                # participant.write_to(outputfile)

                # Write coordinates (only) into another file
                # participant.write_coords_to(coordfile)
                
                # Write coords + affil in a global file
                with open(coordfile, 'a') as cfall:
                    cfall.write("[{0}, {1}, {2}],\n".format(participant.lat, 
                                                            participant.lon,
                                                            participant.affiliation))

                # Append to list
                participantlist.append(participant)
            
    with open(coordfile, 'a') as cf:
        cf.write("]")
cfall.write("]")
logger.info("\n Job finished")
logger.info("{np} participants".format())

INFO:root:Working on file ../data/processed/ParticipantList-1970.tsv
INFO:root:Writing info in file ../data/test2delete/ParticipantList-1970.dat
INFO:root:Writing coordinates in file ../data/test2delete/ParticipantList-1970.dat
INFO:root:----------------
Problem with geocoder <geopy.geocoders.osm.Nominatim object at 0x7fc9a88c49b0>


ValueError: I/O operation on closed file.

In [4]:
participant

NameError: name 'participant' is not defined